In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
#para propresar los datos
import category_encoders as ce #(para pasar los datos categoricos a númericos)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
#para reg lineal y logistica
from sklearn.linear_model import LogisticRegression, LinearRegression
#para reducción de dimensionalidad
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA
#para regresión baggin
import sklearn.metrics as metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
#para regresión boosting
from sklearn.ensemble import GradientBoostingClassifier #(son árboles de decisión como el ramdon forest)
#para árbol de regresión
from sklearn.tree import DecisionTreeRegressor
#para validación
from sklearn.model_selection import cross_val_score, KFold

In [2]:
df_felicidad = pd.read_csv('felicidad.csv')
df_felicidad.head()

,country,rank,score,high,low,gdp,family,lifexp,freedom,generosity,corruption,dystopia
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


In [3]:
df_felicidad['score'].mean()

np.float64(5.354019355774194)

In [4]:
df_felicidad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   country     155 non-null    object 
 1   rank        155 non-null    int64  
 2   score       155 non-null    float64
 3   high        155 non-null    float64
 4   low         155 non-null    float64
 5   gdp         155 non-null    float64
 6   family      155 non-null    float64
 7   lifexp      155 non-null    float64
 8   freedom     155 non-null    float64
 9   generosity  155 non-null    float64
 10  corruption  155 non-null    float64
 11  dystopia    155 non-null    float64
dtypes: float64(10), int64(1), object(1)
memory usage: 14.7+ KB


In [5]:
df_felicidad_01 = df_felicidad.drop(['country','high','low'],axis=1)
df_felicidad_01.head()

,rank,score,gdp,family,lifexp,freedom,generosity,corruption,dystopia
0,1,7.537,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,2,7.522,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,3,7.504,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,4,7.494,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,5,7.469,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


In [6]:
#preparamos la data a utilizar
df_felicidad_features = df_felicidad_01.drop(['score'], axis=1)
df_felicidad_target = df_felicidad_01['score']
x_train, x_test, y_train, y_test = train_test_split(df_felicidad_features, df_felicidad_target, test_size=0.3, random_state=42)

árbol de decisión regresión

con cross_val_score

In [7]:
model = DecisionTreeRegressor()
model.fit(x_train,y_train)
score = cross_val_score(model, df_felicidad_features, df_felicidad_target,cv=5, scoring='neg_mean_squared_error')
#cv sera el número k de particiones con las que se comparara nuestro modelo
#para el caso de clasificación utilizamos scoring ='accuracy'
y_test_pred_model = model.predict(x_test)
print(score)
#interpretamos el score
print(np.abs(score.mean()))

[-0.42155774 -0.09021425 -0.09390944 -0.07781088 -0.41726058]
0.22015057593073983


debemos tomar en cuenta el valor absoluto medio del score comparado convlos valores de  nuestra variable objetivo 'target'

Optimización

In [8]:
#para la optimización
from sklearn.model_selection import RandomizedSearchCV
#el modelo qeu utilizaremos
from sklearn.ensemble import RandomForestRegressor

In [9]:
ran_forest= RandomForestRegressor()

In [13]:
parametros = {
    'n_estimators': range(4,16),
    'criterion': ['squared_error','absolute_error'],
    'max_depth': range(2,11)
}

In [14]:
rand_est = RandomizedSearchCV(estimator=ran_forest, param_distributions=parametros, n_iter=10, cv= 5, scoring='neg_mean_absolute_error', random_state=42)
#el n_inter podemos dejarlo vacio
#recordar que el cv es recomendable en 5 sin embargo se puede variar

In [15]:
#rand_est.fit(x_train,y_train)
rand_est.fit(df_felicidad_features,df_felicidad_target)

/home/paolovf/scikit/env_sci/lib/python3.10/site-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(),
                   param_distributions={'criterion': ['squared_error',
                                                      'absolute_error'],
                                        'max_depth': range(2, 11),
                                        'n_estimators': range(4, 16)},
                   random_state=42, scoring='neg_mean_absolute_error')

In [19]:
rand_est.fit(x_train,y_train)
print('nuestros parametros scogidos son:')
print(rand_est.best_estimator_)

y_test_pred_rand_est = rand_est.predict(x_test)
metrics.mean_absolute_error(y_test,y_test_pred_rand_est)

nuestros parametros scogidos son:
RandomForestRegressor(criterion='absolute_error', max_depth=9, n_estimators=13)


np.float64(0.04498772810310972)

con esto tenemos nuestra mejor esticmación en el rango que utilizamos
lo mismo para otros modelos tanto como de regresiones como de clasificación

Veamos una forma de automatizar de manera automatica
busquemos si esta para wsl de momento solo en linux

In [21]:
##import autosklearn.classification

ModuleNotFoundError: No module named 'autosklearn'